In [8]:
import os
import sys
sys.path.append("../../SpaceFormer/")
device = "cuda"

In [2]:
import scanpy as sc
import squidpy as sq
import pandas as pd
from tqdm.notebook import tqdm
import scipy as sp

In [3]:
import spaceformer as sf

## Process dataset

You can download the dataset [here](https://data.mendeley.com/datasets/mpjzbtfgfr/1).

In [4]:
data = pd.read_csv("E:/codex/CRC_clusters_neighborhoods_markers.csv", index_col=0)
features = [
       'CD44 - stroma:Cyc_2_ch_2', 'FOXP3 - regulatory T cells:Cyc_2_ch_3',
       'CD8 - cytotoxic T cells:Cyc_3_ch_2',
       'p53 - tumor suppressor:Cyc_3_ch_3',
       'GATA3 - Th2 helper T cells:Cyc_3_ch_4',
       'CD45 - hematopoietic cells:Cyc_4_ch_2', 'T-bet - Th1 cells:Cyc_4_ch_3',
       'beta-catenin - Wnt signaling:Cyc_4_ch_4', 'HLA-DR - MHC-II:Cyc_5_ch_2',
       'PD-L1 - checkpoint:Cyc_5_ch_3', 'Ki67 - proliferation:Cyc_5_ch_4',
       'CD45RA - naive T cells:Cyc_6_ch_2', 'CD4 - T helper cells:Cyc_6_ch_3',
       'CD21 - DCs:Cyc_6_ch_4', 'MUC-1 - epithelia:Cyc_7_ch_2',
       'CD30 - costimulator:Cyc_7_ch_3', 'CD2 - T cells:Cyc_7_ch_4',
       'Vimentin - cytoplasm:Cyc_8_ch_2', 'CD20 - B cells:Cyc_8_ch_3',
       'LAG-3 - checkpoint:Cyc_8_ch_4', 'Na-K-ATPase - membranes:Cyc_9_ch_2',
       'CD5 - T cells:Cyc_9_ch_3', 'IDO-1 - metabolism:Cyc_9_ch_4',
       'Cytokeratin - epithelia:Cyc_10_ch_2',
       'CD11b - macrophages:Cyc_10_ch_3', 'CD56 - NK cells:Cyc_10_ch_4',
       'aSMA - smooth muscle:Cyc_11_ch_2', 'BCL-2 - apoptosis:Cyc_11_ch_3',
       'CD25 - IL-2 Ra:Cyc_11_ch_4', 'CD11c - DCs:Cyc_12_ch_3',
       'PD-1 - checkpoint:Cyc_12_ch_4',
       'Granzyme B - cytotoxicity:Cyc_13_ch_2', 'EGFR - signaling:Cyc_13_ch_3',
       'VISTA - costimulator:Cyc_13_ch_4', 'CD15 - granulocytes:Cyc_14_ch_2',
       'ICOS - costimulator:Cyc_14_ch_4',
       'Synaptophysin - neuroendocrine:Cyc_15_ch_3',
       'GFAP - nerves:Cyc_16_ch_2', 'CD7 - T cells:Cyc_16_ch_3',
       'CD3 - T cells:Cyc_16_ch_4',
       'Chromogranin A - neuroendocrine:Cyc_17_ch_2',
       'CD163 - macrophages:Cyc_17_ch_3', 'CD45RO - memory cells:Cyc_18_ch_3',
       'CD68 - macrophages:Cyc_18_ch_4', 'CD31 - vasculature:Cyc_19_ch_3',
       'Podoplanin - lymphatics:Cyc_19_ch_4', 'CD34 - vasculature:Cyc_20_ch_3',
       'CD38 - multifunctional:Cyc_20_ch_4',
       'CD138 - plasma cells:Cyc_21_ch_3', 'HOECHST1:Cyc_1_ch_1',
       'CDX2 - intestinal epithelia:Cyc_2_ch_4',
       'Collagen IV - bas. memb.:Cyc_12_ch_2',
       'CD194 - CCR4 chemokine R:Cyc_14_ch_3',
       'MMP9 - matrix metalloproteinase:Cyc_15_ch_2',
       'CD71 - transferrin R:Cyc_15_ch_4', 'CD57 - NK cells:Cyc_17_ch_4',
       'MMP12 - matrix metalloproteinase:Cyc_21_ch_4', 'DRAQ5:Cyc_23_ch_4']

metadata = data.drop(features, axis=1)
data = data[features]

In [5]:
all_adata = sc.AnnData(data, obs=metadata)
all_adata.obsm['spatial'] = all_adata.obs[['X:X', 'Y:Y']].to_numpy()

adatas = []
for i in all_adata.obs['patients'].unique():
    adatas.append(all_adata[all_adata.obs['patients'] == i])

dataset, label_dict = sf.make_dataset(adatas, 'ClusterName', use_hvg=False)

C:\Users\lshh\miniconda3\envs\py311_torch211_cuda121\Lib\site-packages\anndata\_core\anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
  0%|                                                                                           | 0/35 [00:00<?, ?it/s]C:\Users\lshh\miniconda3\envs\py311_torch211_cuda121\Lib\site-packages\scanpy\preprocessing\_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
C:\Users\lshh\miniconda3\envs\py311_torch211_cuda121\Lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
  3%|██▎                             

In [6]:
label_dict

{'stroma': 0,
 'NK cells': 1,
 'CD4+ T cells GATA3+': 2,
 'CD11b+ monocytes': 3,
 'immune cells': 4,
 'CD11b+CD68+ macrophages': 5,
 'CD163+ macrophages': 6,
 'smooth muscle': 7,
 'vasculature': 8,
 'CD4+ T cells CD45RO+': 9,
 'CD68+ macrophages': 10,
 'dirt': 11,
 'CD8+ T cells': 12,
 'undefined': 13,
 'CD3+ T cells': 14,
 'nerves': 15,
 'B cells': 16,
 'CD4+ T cells': 17,
 'CD68+CD163+ macrophages': 18,
 'lymphatics': 19,
 'tumor cells': 20,
 'adipocytes': 21,
 'tumor cells / immune cells': 22,
 'Tregs': 23,
 'granulocytes': 24,
 'plasma cells': 25,
 'CD68+ macrophages GzmB+': 26,
 'CD11c+ DCs': 27,
 'immune cells / vasculature': 28}

## Create model

In [7]:
sf.set_random_seed(42)
model = sf.SpaceFormer(cell_mask_rate=.5, gene_mask_rate=.3, dropout=.1, input_dim=58, ffn_dim=58, gamma=.2)
model = model.to(device)

## Train model

In [10]:
sf.train(model, dataset, log_dir='log/', max_epoch=20)

[2024-04-01 00:32:09,790::train::INFO] Epoch 1: train_loss 0.8613738162176949
[2024-04-01 00:32:58,218::train::INFO] Epoch 2: train_loss 0.8001658354486738
[2024-04-01 00:33:44,839::train::INFO] Epoch 3: train_loss 0.6219271957874298
[2024-04-01 00:34:33,166::train::INFO] Epoch 4: train_loss 0.4033731345619474
[2024-04-01 00:35:31,805::train::INFO] Epoch 5: train_loss 0.23019901514053345
[2024-04-01 00:36:26,693::train::INFO] Epoch 6: train_loss 0.12075866907835006
[2024-04-01 00:37:16,887::train::INFO] Epoch 7: train_loss 0.06562761898551668
[2024-04-01 00:38:05,693::train::INFO] Epoch 8: train_loss 0.03865141293832234
[2024-04-01 00:38:58,254::train::INFO] Epoch 9: train_loss 0.02611211354711226
[2024-04-01 00:39:47,937::train::INFO] Epoch 10: train_loss 0.018588671833276747
[2024-04-01 00:40:35,121::train::INFO] Epoch 11: train_loss 0.014435682339327675
[2024-04-01 00:41:24,166::train::INFO] Epoch 12: train_loss 0.011988223463829075
[2024-04-01 00:42:10,035::train::INFO] Epoch 13: t